<a href="https://colab.research.google.com/github/Mathias2204/Data_Science_MK/blob/main/Midterm_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 2: Background questions